# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, r2_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df =pd.read_sql("SELECT * FROM Figure8Messages", engine)
df

X = df['message'].tolist()
Y = df.drop(columns=['message', 'original','genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    text =  re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    tokens = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()
    stop = stopwords.words('english')
    
    clean_tokens = []
    for token in tokens:
        if token not in stop:
            clean_token = stemmer.stem(lemmatizer.lemmatize(token))
            clean_tokens.append(clean_token)

    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, train_size=0.7, random_state=42)

#fitting the model
pipeline.fit(x_train,y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
#Predict the values
y_pred = pipeline.predict(x_test)

In [7]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

# display_results(y_test, y_pred)
for i in range(y_test.shape[1]):
    print(y_test.columns[i])
    print('--------------------')
    display_results(y_test.iloc[:,i], y_pred[:,i])
    print('\n\n')

related
--------------------
Labels: [0 1 2]
Confusion Matrix:
 [[ 853 1021   10]
 [ 488 5418   16]
 [   4   38   17]]
Accuracy: 0.799491417673



request
--------------------
Labels: [0 1]
Confusion Matrix:
 [[6381  153]
 [ 732  599]]
Accuracy: 0.887476160203



offer
--------------------
Labels: [0]
Confusion Matrix:
 [[7831]]
Accuracy: 0.995677050223



aid_related
--------------------
Labels: [0 1]
Confusion Matrix:
 [[3918  725]
 [1250 1972]]
Accuracy: 0.74888747616



medical_help
--------------------
Labels: [0 1]
Confusion Matrix:
 [[7166   58]
 [ 560   81]]
Accuracy: 0.921424030515



medical_products
--------------------
Labels: [0 1]
Confusion Matrix:
 [[7444   14]
 [ 375   32]]
Accuracy: 0.950540368722



search_and_rescue
--------------------
Labels: [0 1]
Confusion Matrix:
 [[7662   17]
 [ 170   16]]
Accuracy: 0.976223776224



security
--------------------
Labels: [0 1]
Confusion Matrix:
 [[7718    3]
 [ 144    0]]
Accuracy: 0.981309599491



military
-------------------

### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
pipeline.steps

[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
          ngram_range=(1, 1), preprocessor=None, stop_words=None,
          strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
          tokenizer=<function tokenize at 0x7f45ecebce18>, vocabulary=None)),
 ('tfidf',
  TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
 ('clf',
  MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
              max_depth=None, max_features='auto', max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
              oob_score=False, random_state=None, verbose=0,
              warm_start=False),

In [ ]:
parameters = {
    'vect__max_df': [0.5, 0.75, 1],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators' : [100, 250, 500],
    'clf__estimator__min_samples_leaf': [1, 2, 4,]
} 

cv =  GridSearchCV(pipeline, param_grid=parameters)
cv.fit(x_train, y_train)


In [ ]:
cv.best_estimator_.steps

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_new - cv.predict(x_test)

# display_results(y_test, y_pred)
for i in range(y_test.shape[1]):
    print(y_test.columns[i])
    print('--------------------')
    display_results(y_test.iloc[:,i], y_pred[:,i])
    print('\n\n')

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipelineNB = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(MultinomialNB()))
    ])

parameters = {
    'vect__max_df': [0.5, 0.75, 1],
    'tfidf__sublinear_tf': [True, False],
    'tfidf__use_idf': (True, False),
    'tfidf__min_df' : [1, 2, 5],
    'clf__alpha' : [0.0, 1.0],
} 

cvNB = GridSearchCV(pipelineNB, param_grid=parameters)

cvNB.fit(X_train, y_train)

In [ ]:
y_pred3 = cv.predict(X_test)

# display_results(y_test, y_pred)
for i in range(y_test.shape[1]):
    print(y_test.columns[i])
    print('--------------------')
    display_results(y_test.iloc[:,i], y_pred[:,i])
    print('\n\n')

### 9. Export your model as a pickle file

In [ ]:
from joblib import dump, load
dump(pipeline, 'pipeline.pkl')
dump(cv.best_estimator_, 'cv_best.pkl')

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.